# A small dose of TF internals

In [19]:
import tensorflow as tf
import numpy as np

In [18]:
sess = tf.InteractiveSession()

## Graphs

You can use TensorFlow to build pretty advanced models without ever learning about how to manipulate graphs. But learning a small amount of the graph backend can yield some nice returns in the form of code cleanliness and runtime efficiency. Let's go create a placeholder tensor, which will be placed in the *default graph*:

In [2]:
x = tf.placeholder(tf.float32)

Where is the default graph? Right here:

In [3]:
x.graph

You can access it by this handy helper as well:

In [4]:
tf.get_default_graph()

### Juggling graphs

We can easily make our own custom graph as follows:

In [5]:
g = tf.Graph()

You can use a Python context manager in order to establish your custom graph as the default graph:

In [7]:
with g.as_default():
    g_x = tf.placeholder(tf.float32)

In [9]:
g_x.graph, g_x.graph == x.graph

(<tensorflow.python.framework.ops.Graph at 0x7f6de1c09650>, False)

It is illegal to combine tensors from two different graphs.

In [11]:
g_x + x

ValueError: Tensor("Placeholder:0", dtype=float32) must be from the same graph as Tensor("Placeholder:0", dtype=float32).

Why would we need to juggle graphs?

- Manage separate train / test graphs in the same runtime
- Manage a set of sequence model graphs, each unrolled to a different length

## Introspection

When debugging a hairy TensorFlow graph, it is sometimes useful to dive into the misbehaving op and inspect its local behavior. Here are some basic tools for poking around in a TF graph from inside the code.

In [35]:
x = tf.placeholder(tf.float32, (None,), name="x")

In [36]:
y = tf.placeholder(tf.float32, (None,), name="y")

In [37]:
z = x + y

Here `z` is a `Tensor` object -- i.e., a data edge flowing out of some op (`Operation`) node in our graph.

In [56]:
type(z)

tensorflow.python.framework.ops.Tensor

We can easily access the op which generates `z`:

In [42]:
z.op, z.op.name, z.op.op_def.name

(<tensorflow.python.framework.ops.Operation at 0x7f6de1b33890>,
 u'add_5',
 u'Add')

`op.inputs` contains what you'd expect:

In [46]:
list(z.op.inputs), list(z.op.inputs) == [x, y]

([<tensorflow.python.framework.ops.Tensor at 0x7f6de1b33bd0>,
 True)

We can also introspect in the reverse direction, asking which ops consume the tensor `x`:

In [49]:
x.consumers(), x.consumers()[0] == z.op

([<tensorflow.python.framework.ops.Operation at 0x7f6de1b33890>], True)

This introspection can go to arbitrary depth:

In [51]:
w = x + z

In [53]:
list(w.op.inputs[1].op.inputs), list(w.op.inputs[1].op.inputs) == [x, y]

([<tensorflow.python.framework.ops.Tensor at 0x7f6de1b33bd0>,
 True)

Every op has zero or more outputs (i.e., edges out of the op node):

In [55]:
w.op.outputs, w.op.outputs[0] == w

([<tensorflow.python.framework.ops.Tensor at 0x7f6de1b9dd10>], True)

## Variable updates

In [58]:
W = tf.Variable(np.zeros((10,)), name="W")

In [60]:
updated_W = tf.assign(W, np.ones((10,)))

In [61]:
type(updated_W)

tensorflow.python.framework.ops.Tensor

In [62]:
updated_W.op, updated_W.op.name, updated_W.op.op_def.name

(<tensorflow.python.framework.ops.Operation at 0x7f6de1b9de90>,
 u'Assign',
 u'Assign')

In [65]:
list(updated_W.op.inputs), [x.name for x in list(updated_W.op.inputs)]

([<tensorflow.python.framework.ops.Tensor at 0x7f6de1b16390>,
 [u'W:0', u'Assign/value:0'])

In [67]:
updated_W.op.outputs, updated_W.op.outputs == [updated_W]

([<tensorflow.python.framework.ops.Tensor at 0x7f6de1b16350>], True)

## Going deeper: finding an `Op` implementation

Every TensorFlow Op has one or more **kernels**: architecture-specific implementations in C++ or CUDA.

At package compile-time, TensorFlow generates a bunch of function stubs which allow for easy access to these kernels. Unfortunately, they make understanding the backend a bit more difficult.

If you go on a naive hunt for an op implementation, you'll run into one of these stubs. For example, here's the stub for `tf.gather`:

In [72]:
tf.gather??

```
Signature: tf.gather(params, indices, name=None)
Source:
def gather(params, indices, name=None):
  r"""Gather slices from `params` according to `indices`.
  
  -- snip --
  """
  return _op_def_lib.apply_op("Gather", params=params, indices=indices,
                              name=name)

File:      /.../tensorflow/python/ops/gen_array_ops.py
Type:      function
```

Pretty unsatisfying. Notice the file is `gen_array_ops.py`. If you open this file in your local TF installation, you'll find it's full of boring stubs like this. (You won't find these generated stub files on the TF GitHub, of course.)

The kernel code you might care about actually lives in [`tensorflow/core/kernels`](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/core/kernels). There is not a one-to-one mapping between kernel files and the generated stub files, so you'll need to resort to `grep`ping often.

For `tf.gather`, the kernel implementation happens to live in [`tensorflow/core/kernels/gather_op.cc`](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/kernels/gather_op.cc).

For ops which have a GPU kernel implementation as well, you can find it in a pretty clearly-named file in the same folder. For example, the `tf.concat` operation has a GPU implementation at [`tensorflow/core/kernels/concat_op_gpu.cu.cc`](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/kernels/concat_op_gpu.cu.cc).